In [2]:
!pip3 install tensorflow-serving-api==2.8.0 opencv-python mediapipe matplotlib scikit-learn

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_styled_landmarks(image, results):
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 
    


In [4]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
#         print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [4]:
def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

In [18]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['openPlam', 'closeFist', 'openPlamToRight','openPlamToLeft'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 30

In [19]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [13]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
#                 print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()


KeyboardInterrupt: 

In [14]:
cap.release()
cv2.destroyAllWindows()

In [20]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [21]:
label_map = {label:num for num, label in enumerate(actions)}

In [22]:
label_map

{'openPlam': 0, 'closeFist': 1, 'openPlamToRight': 2, 'openPlamToLeft': 3}

In [23]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [24]:
np.array(sequences).shape

(120, 30, 126)

In [25]:
np.array(labels).shape

(120,)

In [26]:
X = np.array(sequences)

In [27]:
X.shape

(120, 30, 126)

In [28]:
y = to_categorical(labels).astype(int)

In [29]:
y

array([[1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1,

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [32]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [33]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,126)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [34]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [23]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])


Epoch 1/2000


4/4 [==============================] - 4s 23ms/step - loss: 1.3760 - categorical_accuracy: 0.2456
Epoch 2/2000
4/4 [==============================] - 0s 21ms/step - loss: 1.3104 - categorical_accuracy: 0.2544
Epoch 3/2000
4/4 [==============================] - 0s 21ms/step - loss: 1.8194 - categorical_accuracy: 0.2544
Epoch 4/2000
4/4 [==============================] - 0s 21ms/step - loss: 1.2069 - categorical_accuracy: 0.4298
Epoch 5/2000
4/4 [==============================] - 0s 21ms/step - loss: 1.1304 - categorical_accuracy: 0.5614
Epoch 6/2000
4/4 [==============================] - 0s 21ms/step - loss: 1.0954 - categorical_accuracy: 0.5877
Epoch 7/2000
4/4 [==============================] - 0s 21ms/step - loss: 1.0449 - categorical_accuracy: 0.5789
Epoch 8/2000
4/4 [==============================] - 0s 21ms/step - loss: 1.0022 - categorical_accuracy: 0.5877
Epoch 9/2000
4/4 [==============================] - 0s 22ms/step - loss: 0.9338 - categorical_accuracy: 0.6140

4/4 [==============================] - 0s 21ms/step - loss: 0.2291 - categorical_accuracy: 0.9474
Epoch 142/2000
4/4 [==============================] - 0s 21ms/step - loss: 0.2086 - categorical_accuracy: 0.9474
Epoch 143/2000
4/4 [==============================] - 0s 20ms/step - loss: 0.1960 - categorical_accuracy: 0.9474
Epoch 144/2000
4/4 [==============================] - 0s 20ms/step - loss: 0.3592 - categorical_accuracy: 0.9211
Epoch 145/2000
4/4 [==============================] - 0s 20ms/step - loss: 0.3555 - categorical_accuracy: 0.8947
Epoch 146/2000
4/4 [==============================] - 0s 21ms/step - loss: 0.2597 - categorical_accuracy: 0.8684
Epoch 147/2000
4/4 [==============================] - 0s 20ms/step - loss: 0.2742 - categorical_accuracy: 0.9298
Epoch 148/2000
4/4 [==============================] - 0s 20ms/step - loss: 0.2200 - categorical_accuracy: 0.9561
Epoch 149/2000
4/4 [==============================] - 0s 21ms/step - loss: 0.2840 - categorical_accuracy: 0.947

4/4 [==============================] - 0s 23ms/step - loss: 0.7751 - categorical_accuracy: 0.7018
Epoch 286/2000
4/4 [==============================] - 0s 24ms/step - loss: 0.7663 - categorical_accuracy: 0.7105
Epoch 287/2000
4/4 [==============================] - 0s 24ms/step - loss: 0.7595 - categorical_accuracy: 0.7193
Epoch 288/2000
4/4 [==============================] - 0s 22ms/step - loss: 0.7532 - categorical_accuracy: 0.7018
Epoch 289/2000
4/4 [==============================] - 0s 21ms/step - loss: 0.7483 - categorical_accuracy: 0.7018
Epoch 290/2000
4/4 [==============================] - 0s 21ms/step - loss: 0.7431 - categorical_accuracy: 0.7368
Epoch 291/2000
4/4 [==============================] - 0s 24ms/step - loss: 0.7376 - categorical_accuracy: 0.7281
Epoch 292/2000
4/4 [==============================] - 0s 24ms/step - loss: 0.7301 - categorical_accuracy: 0.7281
Epoch 293/2000
4/4 [==============================] - 0s 24ms/step - loss: 0.7240 - categorical_accuracy: 0.728

4/4 [==============================] - 0s 22ms/step - loss: 0.7245 - categorical_accuracy: 0.6754
Epoch 430/2000
4/4 [==============================] - 0s 21ms/step - loss: 0.7111 - categorical_accuracy: 0.7018
Epoch 431/2000
4/4 [==============================] - 0s 22ms/step - loss: 0.7108 - categorical_accuracy: 0.6930
Epoch 432/2000
4/4 [==============================] - 0s 21ms/step - loss: 0.7255 - categorical_accuracy: 0.6667
Epoch 433/2000
4/4 [==============================] - 0s 21ms/step - loss: 1.3356 - categorical_accuracy: 0.6404
Epoch 434/2000
4/4 [==============================] - 0s 21ms/step - loss: 0.7388 - categorical_accuracy: 0.6579
Epoch 435/2000
4/4 [==============================] - 0s 21ms/step - loss: 0.7507 - categorical_accuracy: 0.6667
Epoch 436/2000
4/4 [==============================] - 0s 21ms/step - loss: 0.8240 - categorical_accuracy: 0.6754
Epoch 437/2000
4/4 [==============================] - 0s 21ms/step - loss: 0.7624 - categorical_accuracy: 0.701

4/4 [==============================] - 0s 20ms/step - loss: 0.7938 - categorical_accuracy: 0.6579
Epoch 574/2000
4/4 [==============================] - 0s 20ms/step - loss: 0.7740 - categorical_accuracy: 0.6404
Epoch 575/2000
4/4 [==============================] - 0s 22ms/step - loss: 0.7630 - categorical_accuracy: 0.6140
Epoch 576/2000
4/4 [==============================] - 0s 21ms/step - loss: 0.7464 - categorical_accuracy: 0.6579
Epoch 577/2000
4/4 [==============================] - 0s 21ms/step - loss: 0.7162 - categorical_accuracy: 0.6404
Epoch 578/2000
4/4 [==============================] - 0s 20ms/step - loss: 0.7383 - categorical_accuracy: 0.6667
Epoch 579/2000
4/4 [==============================] - 0s 21ms/step - loss: 0.7451 - categorical_accuracy: 0.6404
Epoch 580/2000
4/4 [==============================] - 0s 22ms/step - loss: 0.7831 - categorical_accuracy: 0.6316
Epoch 581/2000
4/4 [==============================] - 0s 22ms/step - loss: 0.6557 - categorical_accuracy: 0.684

...

4/4 [==============================] - 0s 21ms/step - loss: 0.0088 - categorical_accuracy: 1.0000
Epoch 1510/2000
4/4 [==============================] - 0s 21ms/step - loss: 0.0083 - categorical_accuracy: 1.0000
Epoch 1511/2000
4/4 [==============================] - 0s 21ms/step - loss: 0.0080 - categorical_accuracy: 1.0000
Epoch 1512/2000
4/4 [==============================] - 0s 20ms/step - loss: 0.0078 - categorical_accuracy: 1.0000
Epoch 1513/2000
4/4 [==============================] - 0s 21ms/step - loss: 0.0075 - categorical_accuracy: 1.0000
Epoch 1514/2000
4/4 [==============================] - 0s 20ms/step - loss: 0.0072 - categorical_accuracy: 1.0000
Epoch 1515/2000
4/4 [==============================] - 0s 20ms/step - loss: 0.0070 - categorical_accuracy: 1.0000
Epoch 1516/2000
4/4 [==============================] - 0s 21ms/step - loss: 0.0071 - categorical_accuracy: 1.0000
Epoch 1517/2000
4/4 [==============================] - 0s 20ms/step - loss: 0.0093 - categorical_accurac

4/4 [==============================] - 0s 20ms/step - loss: 0.0021 - categorical_accuracy: 1.0000
Epoch 1582/2000
4/4 [==============================] - 0s 20ms/step - loss: 0.0021 - categorical_accuracy: 1.0000
Epoch 1583/2000
4/4 [==============================] - 0s 20ms/step - loss: 0.0021 - categorical_accuracy: 1.0000
Epoch 1584/2000
4/4 [==============================] - 0s 20ms/step - loss: 0.0020 - categorical_accuracy: 1.0000
Epoch 1585/2000
4/4 [==============================] - 0s 21ms/step - loss: 0.0020 - categorical_accuracy: 1.0000
Epoch 1586/2000
4/4 [==============================] - 0s 20ms/step - loss: 0.0020 - categorical_accuracy: 1.0000
Epoch 1587/2000
4/4 [==============================] - 0s 20ms/step - loss: 0.0020 - categorical_accuracy: 1.0000
Epoch 1588/2000
4/4 [==============================] - 0s 20ms/step - loss: 0.0019 - categorical_accuracy: 1.0000
Epoch 1589/2000
4/4 [==============================] - 0s 21ms/step - loss: 0.0019 - categorical_accurac

4/4 [==============================] - 0s 20ms/step - loss: 0.0010 - categorical_accuracy: 1.0000
Epoch 1654/2000
4/4 [==============================] - 0s 21ms/step - loss: 0.0010 - categorical_accuracy: 1.0000
Epoch 1655/2000
4/4 [==============================] - 0s 21ms/step - loss: 0.0010 - categorical_accuracy: 1.0000
Epoch 1656/2000
4/4 [==============================] - 0s 21ms/step - loss: 9.9991e-04 - categorical_accuracy: 1.0000
Epoch 1657/2000
4/4 [==============================] - 0s 20ms/step - loss: 9.8079e-04 - categorical_accuracy: 1.0000
Epoch 1658/2000
4/4 [==============================] - 0s 20ms/step - loss: 9.7120e-04 - categorical_accuracy: 1.0000
Epoch 1659/2000
4/4 [==============================] - 0s 20ms/step - loss: 9.6391e-04 - categorical_accuracy: 1.0000
Epoch 1660/2000
4/4 [==============================] - 0s 21ms/step - loss: 9.5195e-04 - categorical_accuracy: 1.0000
Epoch 1661/2000
4/4 [==============================] - 0s 21ms/step - loss: 9.4239e-

4/4 [==============================] - 0s 21ms/step - loss: 3.8669e-04 - categorical_accuracy: 1.0000
Epoch 1792/2000
4/4 [==============================] - 0s 20ms/step - loss: 3.8386e-04 - categorical_accuracy: 1.0000
Epoch 1793/2000
4/4 [==============================] - 0s 21ms/step - loss: 3.8109e-04 - categorical_accuracy: 1.0000
Epoch 1794/2000
4/4 [==============================] - 0s 21ms/step - loss: 3.7912e-04 - categorical_accuracy: 1.0000
Epoch 1795/2000
4/4 [==============================] - 0s 21ms/step - loss: 3.7589e-04 - categorical_accuracy: 1.0000
Epoch 1796/2000
4/4 [==============================] - 0s 21ms/step - loss: 3.7550e-04 - categorical_accuracy: 1.0000
Epoch 1797/2000
4/4 [==============================] - 0s 20ms/step - loss: 3.7304e-04 - categorical_accuracy: 1.0000
Epoch 1798/2000
4/4 [==============================] - 0s 20ms/step - loss: 3.7025e-04 - categorical_accuracy: 1.0000
Epoch 1799/2000
4/4 [==============================] - 0s 22ms/step - lo

4/4 [==============================] - 0s 21ms/step - loss: 1.8576e-04 - categorical_accuracy: 1.0000
Epoch 1930/2000
4/4 [==============================] - 0s 21ms/step - loss: 1.8465e-04 - categorical_accuracy: 1.0000
Epoch 1931/2000
4/4 [==============================] - 0s 21ms/step - loss: 1.8408e-04 - categorical_accuracy: 1.0000
Epoch 1932/2000
4/4 [==============================] - 0s 21ms/step - loss: 1.8318e-04 - categorical_accuracy: 1.0000
Epoch 1933/2000
4/4 [==============================] - 0s 21ms/step - loss: 1.8235e-04 - categorical_accuracy: 1.0000
Epoch 1934/2000
4/4 [==============================] - 0s 21ms/step - loss: 1.8169e-04 - categorical_accuracy: 1.0000
Epoch 1935/2000
4/4 [==============================] - 0s 21ms/step - loss: 1.8056e-04 - categorical_accuracy: 1.0000
Epoch 1936/2000
4/4 [==============================] - 0s 22ms/step - loss: 1.7952e-04 - categorical_accuracy: 1.0000
Epoch 1937/2000
4/4 [==============================] - 0s 21ms/step - lo

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            48896     
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 4)                 132       
                                                                 
Total params: 203492 (794.89 KB)
Trainable params: 20349

In [35]:
res = model.predict(X_test)

1/1 [==============================] - 1s 706ms/step


In [36]:
actions[np.argmax(res[1])]

'openPlamToLeft'

In [37]:
actions[np.argmax(y_test[3])]

'openPlamToLeft'

In [38]:
model.save('action2.hdf5')

C:\Users\ASUS\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [39]:
model.load_weights('action.hdf5')

In [40]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [41]:
yhat = model.predict(X_test)

1/1 [==============================] - 0s 22ms/step


In [42]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [43]:
ytrue

[2, 1, 3, 3, 1, 1]

In [44]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[3, 0],
        [0, 3]],

       [[5, 0],
        [0, 1]],

       [[4, 0],
        [0, 2]]], dtype=int64)

In [45]:
accuracy_score(ytrue, yhat)

1.0

In [38]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [1]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.4

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.insert(0,keypoints)
#         sequence = sequence[:30]
#         sequence.append(keypoints)
        sequence = sequence[:30]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
#             image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

NameError: name 'cv2' is not defined

In [50]:
res[np.argmax(res)]>threshold

True

In [5]:
cap.release()
cv2.destroyAllWindows()

In [34]:
model.predict(X_test)

1/1 [==============================] - 0s 21ms/step


array([[3.4752092e-16, 2.1638441e-20, 2.1977431e-09, 1.0000000e+00],
       [1.7463007e-10, 1.2060391e-12, 5.7034296e-08, 1.0000000e+00],
       [1.5971119e-06, 9.9102392e-18, 1.1880891e-12, 9.9999845e-01],
       [1.0000000e+00, 2.2196254e-19, 0.0000000e+00, 0.0000000e+00],
       [1.8349397e-08, 1.0000000e+00, 8.9058643e-16, 7.2322148e-17],
       [1.8581875e-14, 8.8264941e-15, 9.9998915e-01, 1.0788453e-05]],
      dtype=float32)

In [35]:
model.predict(np.expand_dims(X_test[0], axis=0))

1/1 [==============================] - 0s 22ms/step


array([[3.4752092e-16, 2.1638441e-20, 2.1977431e-09, 1.0000000e+00]],
      dtype=float32)

In [3]:
import sys
print(sys.version)




3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]
